In [10]:
%pip install newsapi-python
%pip install requests
%pip install newsapi-python
%pip install matplotlib
%pip install pandas
%pip install bs4 
%pip install numpy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring instal

In [10]:
import pprint
import requests
from newsapi import NewsApiClient
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime as dt
from datetime import timedelta

secret = '629525a47ff6458789d19a3c489419fa'
  
# Define the endpoint
url = 'https://newsapi.org/v2/everything'


# Get today's date
today = datetime.date.today()

# Get the date 2 years ago
two_years = today - datetime.timedelta(days=30)

In [18]:
def get_articlesplayer(player, language, date_to):

    url = 'https://newsapi.org/v2/everything'

    # Create date_form and to for the last month
    today = datetime.date.today()
    date_from = today.strftime("%Y-%m-%d")


    # Specify the query and number of returns
    parameters_bakker = {
        'q': player, # query phrase
        'pageSize': 100,  # maximum is 100
        'apiKey': secret, # your own API key
        'sortBy':'publishedAt',
        'from': date_from,
        'to': date_to, 
        'language': language
    }

    response = requests.get(url, params = parameters_bakker)

    # Parse the JSON response and extract article URLs
    if response.ok:
        data = response.json()
        urls = [article['url'] for article in data['articles']]
    else:
        print('Error: Request failed with status', response.status_code)

    # Scrape the full article content for each URL and store in a dataframe
    articles = []
    for url in urls:
        # Make a request to the article URL
        response = requests.get(url)

        # Parse the HTML response and extract the article content
        if response.ok:
            soup = BeautifulSoup(response.content, 'html.parser')
        
            content = '\n'.join([p.text.strip() for p in soup.find_all('p')])
            articles.append({'content': content})
        else:
            print('Error: Request failed with status', response.status_code)


    # Convert the list of articles to a Pandas dataframe
    df = pd.DataFrame(articles)

    return(articles)




In [19]:
def get_players_df(playerlist, languagelist, date_to):

    df_all_players = pd.DataFrame(columns=['data', 'player', 'language'])

    # Loop over players and given languages
    for player in playerlist:

        for language in languagelist:

            articles_arr = get_articlesplayer(player, language, date_to)

            language_arr = np.full((len(articles_arr)), language)

            player_arr = np.full((len(articles_arr)), player)

            df_one_lang = pd.DataFrame({'data': articles_arr, 'player': player_arr,'language': language_arr})

            df_all_players = pd.concat([df_all_players, df_one_lang])

    return df_all_players



Next Function not finished already

In [5]:
def get_date_of_file(file):
    # get the last time the file was updated 
    creation_time = os.stat(file)
    last_modified_time = creation_time.st_mtime
    seconds = int(last_modified_time)
    microseconds = int((last_modified_time - seconds) * 1000000)
    dta = dt.fromtimestamp(seconds).replace(microsecond=microseconds)
    date_string = dta.strftime('%m-%d-%Y')
    date_obj = dta.strptime(date_string, '%m-%d-%Y')
    # Add one day to the datetime object
    new_date_obj = date_obj + timedelta(days=1)

    # Convert the new datetime object to a string in the desired format
    date_to = new_date_obj.strftime('%m-%d-%Y')

    return date_to


In [26]:
def updateplayerdata(file):
    playerlist = ['Mitchel Bakker', 'Jeremie Frimpong', 'Moussa Diaby', 'Jonathan Tah', 'Piero Hincapie', 'Exequiel Palacios', 'Mykhaylo Mudryk'] 
    languagelist = ['en', 'de', 'es']
    df_new = get_players_df(playerlist, languagelist, get_date_of_file(file))

    # Open the CSV file in append mode and write the DataFrame
    with open(file, 'a', newline='') as f:
        df_new.to_csv(f, header=f.tell()==0, index=False)

    return df_new

In [27]:
file = '/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/data_files/all_data_v2.csv'

updateplayerdata(file)

,data,player,language
